# Create the model from a pre-trained network

__DISCLAIMER__: This notebook is based on the course **Practical Deep Learning for Coders** (http://course.fast.ai/index.html), and is intended as a learning scratchbook for _me_

In [1]:
%matplotlib inline

In [21]:
import os, json
from matplotlib import pyplot as plt 
import libs.utils; reload(libs.utils)
import libs.vgg16; reload(libs.vgg16)
from libs.vgg16 import Vgg16
from libs.utils import plots

In [3]:
#data_path = 'data/dogscats/'
data_path = 'data/dogscats/sample/'
train_path = data_path + 'train/'
valid_path = data_path + 'valid/'
test_path = data_path + 'test1/'
batch_size = 4

In [11]:
vgg = Vgg16()

/Users/stefano/anaconda/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_2 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 3, 224, 224)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


KeyboardInterrupt: 

In [ ]:
batches = vgg.get_batches(train_path, batch_size=batch_size)
val_batches = vgg.get_batches(valid_path, batch_size=batch_size)

In [ ]:
imgs, lab = next(batches)
plots(imgs, titles=lab)

In [ ]:
vgg.predict(imgs, True)

In [ ]:
vgg.classes[:4]

In [ ]:
vgg.finetune(batches)

In [ ]:
vgg.fit(batches, val_batches, nb_epoch=1)

# Create the model from scratch

In [47]:
import numpy as np
from numpy.random import random, permutation
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.models import Sequential, Model
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers import Input
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, RMSprop
from keras.preprocessing import image

In [42]:
FILES_PATH = 'http://www.platform.ai/models/'; CLASS_FILE='imagenet_class_index.json'
fpath = get_file(CLASS_FILE, FILES_PATH+CLASS_FILE, cache_subdir='models')

In [43]:
with open(fpath) as f: 
    class_dict = json.load(f)
    classes = [class_dict[str(i)][1] for i in range(len(class_dict))]

Convolutional Block definition

In [44]:
def ConvBlock(layers, model, filters):
    for l in range(layers):
        model.add(ZeroPadding2D((1,1)))
        model.add(Convolution2D(filters, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

Fully Connected Block definition

In [45]:
def FCBlock(model):
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))

In [48]:
# Mean of each channel as provided by VGG researchers
vgg_mean = np.array([123.68, 116.779, 103.939]).reshape((3,1,1))

def vgg_preprocess(x):
    x = x - vgg_mean     # subtract mean
    return x[:, ::-1]    # reverse axis bgr->rgb

In [53]:
def VGG_16():
    model = Sequential()
    model.add(Lambda(vgg_preprocess, input_shape(3, 224, 224)))
    
    ConvBlock(2, model, 64)
    ConvBlock(2, model, 128)
    ConvBlock(3, model, 256)
    ConvBlock(3, model, 512)
    ConvBlock(3, model, 512)
    
    model.add(Flatten())
    FCBlock(model)
    FCBlock(model)
    model.add(Dense(1000, activation='softmax'))
    return model